In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense

In [26]:
lines = pd.read_table("mar.txt",names = ['mar','random'])
lines.head()

,mar,random
Go.,जा.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
Run!,पळ!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
Run!,धाव!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
Run!,पळा!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
Run!,धावा!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [27]:
lines.reset_index(inplace = True)
lines = lines.drop(['random'], axis = 1)
lines = lines.rename(columns={"index":"eng"})

In [4]:
type(lines)
lines.head()

,eng,mar
0,Go.,जा.
1,Run!,पळ!
2,Run!,धाव!
3,Run!,पळा!
4,Run!,धावा!


In [28]:
# Lowercase all the characters
lines.eng = lines.eng.apply(lambda x: x.lower())
lines.mar = lines.mar.apply(lambda x: x.lower())

# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.mar=lines.mar.apply(lambda x: re.sub("'", '', x))

exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.mar=lines.mar.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.mar = lines.mar.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.mar=lines.mar.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.mar=lines.mar.apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines.mar = lines.mar.apply(lambda x : 'START_ '+ x + ' _END')


In [19]:
lines.sample(10)

,eng,mar
22640,ive spent all the money,START_ startमी सगळा पैसा खर्च केला आहेend _END
3798,tom wont come,START_ startटॉम नाही येणारend _END
24550,its harder than it looks,START_ startवाटतं त्यापेक्षा कठीण आहेend _END
35677,both tom and i are in the hospital,START_ startटॉम आणि मी दोघेही रुग्णालयात आहोतe...
8890,what do you teach,START_ startकाय शिकवतेस तूend _END
39143,many people have asked us that question,START_ startतो प्रश्न आम्हाला अनेक जणांनी विचा...
27986,im very proud of my father,START_ startमला माझ्या वडिलांचा अभिमान वाटतोen...
14851,the house is burning,START_ startते घर जळतंयend _END
41186,who is the current representative from france,START_ startफ्रान्सचा सध्या प्रतिनिधी कोण आहेe...
9830,i still want to go,START_ startमला अजूनही जायचं आहेend _END


In [29]:
#Vocabulary of english
all_eng_words = set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

#Vocabulary of marathi
all_mar_words = set()
for mar in lines.mar:
    for word in mar.split():
        if word not in all_mar_words:
            all_mar_words.add(word)

In [30]:
all_eng_words

{'starve',
 'expected',
 'earthquakes',
 'joke',
 'ozaki',
 'failed',
 'talk',
 'boxer',
 'commercial',
 'noodles',
 'articles',
 'fired',
 'vibrating',
 'strike',
 'teacher',
 'preserved',
 'future',
 'canoe',
 'windy',
 'pop',
 'appointment',
 'poem',
 'yesterdays',
 'sorry',
 'note',
 'funny',
 'real',
 'cannot',
 'similar',
 'farmer',
 'to',
 'online',
 'australian',
 'pass',
 'khalifa',
 'isaac',
 'kites',
 'pages',
 'maintenance',
 'theirs',
 'straight',
 'villagers',
 'mark',
 'experiment',
 'omelette',
 'germany',
 'powerful',
 'dart',
 'youtube',
 'prague',
 'cuba',
 'youve',
 'conservative',
 'id',
 'fortunate',
 'rescue',
 'height',
 'atone',
 'accent',
 'kimonos',
 'theyll',
 'yearold',
 'shiny',
 'isopropyl',
 'brick',
 'harpsichord',
 'digitalis',
 'warsaw',
 'blackmailed',
 'destroy',
 'upside',
 'urgent',
 'cancelled',
 'amiens',
 'nuggets',
 'castles',
 'youngest',
 'mahjong',
 'matsuyamas',
 'bled',
 'bus',
 'forehead',
 'nails',
 'against',
 'dam',
 'accordion',
 'si

In [31]:
all_mar_words

{'नदीच्या',
 'कसलीही',
 'कॉमिक',
 'बँडविड्थ',
 'उन्होंने',
 'हसत',
 'ओरडू',
 'यूरोपमध्ये',
 'घातलंस',
 'टेक्सन',
 'समजायचं',
 'त्याच्यापेक्षा',
 'प्यावंसं',
 'विनोदवीर',
 'ब्राजिलमध्ये',
 'दहाव्या',
 'झाली',
 'पक्षी',
 'वर्षांचे',
 'टूरच्या',
 'रहायला',
 'इशारा',
 'रसत्यात',
 'बसलोय',
 'सुरक्षित',
 'लिहायची',
 'शेवट',
 'स्त्रियांपेक्षा',
 'पोहोचायला',
 'टॉमवरती',
 'मित्राच्या',
 'आखात',
 'नेसी',
 'करिमाने',
 'इंडोयुरोपीय',
 'हॅलोवीन',
 'पेटवल्या',
 'असणार',
 'ठेवल्या',
 'भरत',
 'आत्मचरित्र',
 'अमेरिकनांसारखेच',
 'मिनिटांसाठी',
 'लग्न',
 'टेस्ट',
 'विद्यार्थिनीची',
 'निवडताना',
 'पॅकिंगला',
 'हेतूने',
 'घालवते',
 'फार्मसीचं',
 'मधोमध',
 'भाषेपासून',
 'बर्फाबरोबर',
 'सर्वांसाठी',
 'जिने',
 'मागतात',
 'गमावले',
 'स्वातंत्र्यदिन',
 'प्रोसेसरचा',
 'छोट्या',
 'धरलेली',
 'जीएमटीपर्यंत',
 'मार्गारिनमधला',
 'भुकेले',
 'द्वीप',
 'कौतुक',
 'परभाषा',
 'नावाने',
 'कार्डबरोबर',
 'पाळायचा',
 'घडूच',
 'गळे',
 'मुलांनी',
 'सेंटिमीटर',
 'संपवणं',
 'डार्ट',
 'देशाचा',
 'मॅनेजर',
 'मागच्या',
 'राज्यावर',


In [32]:
length_list = []
for l in lines.eng:
    length_list.append(len(l.split(" ")))
max_length_tar = np.max(length_list)
max_length_tar

34

In [33]:
length_list = []
for l in lines.mar:
    length_list.append(len(l.split(" ")))
max_length_tar = np.max(length_list)
max_length_tar

37

In [34]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_mar_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_mar_words)
num_encoder_tokens, num_decoder_tokens

(5779, 13975)

In [35]:
num_decoder_tokens += 1
num_decoder_tokens

13976

In [36]:
num_encoder_tokens += 1
num_encoder_tokens

5780

In [41]:
input_token_index = dict([(word, i+1) for i , word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i , word in enumerate(target_words)])

In [42]:
reverse_input_char_index = dict(((i,word) for word, i in input_token_index.items()))
reverse_target_char_index = dict(((i,word) for word, i in target_token_index.items()))

In [43]:
input_token_index

{'a': 1,
 'abbreviation': 2,
 'ability': 3,
 'able': 4,
 'about': 5,
 'above': 6,
 'abroad': 7,
 'absent': 8,
 'absolute': 9,
 'absolutely': 10,
 'absorbed': 11,
 'accent': 12,
 'accept': 13,
 'acceptable': 14,
 'accepted': 15,
 'access': 16,
 'accessible': 17,
 'accident': 18,
 'accidentally': 19,
 'accomplices': 20,
 'according': 21,
 'accordion': 22,
 'account': 23,
 'accused': 24,
 'accusing': 25,
 'accustomed': 26,
 'ace': 27,
 'aces': 28,
 'acetone': 29,
 'ache': 30,
 'aches': 31,
 'achille': 32,
 'achilles': 33,
 'aching': 34,
 'acid': 35,
 'acknowledged': 36,
 'acquired': 37,
 'acres': 38,
 'across': 39,
 'act': 40,
 'acted': 41,
 'acting': 42,
 'action': 43,
 'activist': 44,
 'actor': 45,
 'actors': 46,
 'actress': 47,
 'acts': 48,
 'actual': 49,
 'actually': 50,
 'add': 51,
 'added': 52,
 'addict': 53,
 'addicted': 54,
 'addition': 55,
 'additional': 56,
 'addon': 57,
 'address': 58,
 'adds': 59,
 'administrative': 60,
 'admire': 61,
 'admit': 62,
 'admitted': 63,
 'adopt': 6

In [44]:
target_token_index

{'10': 1,
 '100': 2,
 '300': 3,
 '3ds': 4,
 'START_': 5,
 '_END': 6,
 'a': 7,
 'b': 8,
 'beispiel': 9,
 'h₂o': 10,
 'ntt': 11,
 'tatoebaorg': 12,
 'uk': 13,
 'अ': 14,
 'अँकरेजमार्गे': 15,
 'अँजिलीस': 16,
 'अँटीव्हायरस': 17,
 'अँड': 18,
 'अँडी': 19,
 'अँडीजमधल्या': 20,
 'अँब्युलन्स': 21,
 'अंक': 22,
 'अंकांनी': 23,
 'अंकारा': 24,
 'अंकारात': 25,
 'अंग': 26,
 'अंगठ्या': 27,
 'अंगरक्षकांची': 28,
 'अंगाला': 29,
 'अंगोला': 30,
 'अंघोळ': 31,
 'अंठ्यांपेक्षा': 32,
 'अंड': 33,
 'अंडरवुडचा': 34,
 'अंडरवेअर': 35,
 'अंडी': 36,
 'अंडींचा': 37,
 'अंडीसुद्धा': 38,
 'अंड्यांवर': 39,
 'अंत': 40,
 'अंतर': 41,
 'अंतराळयात्री': 42,
 'अंतराळयान': 43,
 'अंतराळयानाचं': 44,
 'अंतराळयानातून': 45,
 'अंतराळवीर': 46,
 'अंतराळात': 47,
 'अंतरित': 48,
 'अंतिम': 49,
 'अंदाज': 50,
 'अंदाजे': 51,
 'अंधार': 52,
 'अंधाराची': 53,
 'अंधारात': 54,
 'अंमलात': 55,
 'अंशांकित': 56,
 'अकरा': 57,
 'अकरावीत': 58,
 'अकल्पनीय': 59,
 'अकाउंट': 60,
 'अकाउंटमध्ये': 61,
 'अकॉर्डियन': 62,
 'अक्कलदाढ': 63,
 'अक्वॅरियममध्ये': 64,
 'अक्षर